In [10]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
import torch
import os
import json
from datasets import Dataset
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import os
import numpy as np
from transformers import LayoutLMv2Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import default_collate
from torchvision.transforms import ToTensor
import torch.nn as nn
from transformers import LayoutLMv2Model
from transformers.models.layoutlm import LayoutLMConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torchvision
from torchvision.ops import RoIAlign
from transformers import LayoutLMv2Tokenizer
from torchnlp.nn import Attention
import time
from torch.utils.data import DataLoader
from transformers import AdamW
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler,DataLoader,TensorDataset
import pprint
import numpy
from os import listdir
from os.path import join, isfile
dataset_path_train = "/home/pushpa/cord_original/train/image"
dataset_path_test = "/home/pushpa/cord_original/test/image"
labels = set()

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from os import listdir
path = '/home/pushpa/cord_original'
from os.path import isfile, join
onlyfiles = [f for f in listdir(path+'/train/changed') if isfile(join(path+'/train/changed', f))]





for filename in onlyfiles:
    #print(filename)
    annotationpath = path + "/train/changed/"+ filename
    f = open(annotationpath, encoding="utf8")
    data = json.load(f)
    for item in data:
      # print(item)
      labels.add(item['label'])

labels = list(labels)

idx2label = {v: k for v, k in enumerate(labels)}
label2idx = {k: v for v, k in enumerate(labels)}

print(label2idx)

def normalize_box(box, width, height):
     return [
         int(100 * (box[0] / width)),
         int(100 * (box[1] / height)),
         int(100 * (box[2] / width)),
         int(100 * (box[3] / height)),
     ]
# In[2]:
# Create dataframe containing image path
dirpath = path
def create_img_path(dataset_path):
    images_train = []
    dataset_fldr= dataset_path.split("/")[-2]
    for label_folder, _, file_names in os.walk(dataset_path):
      
      for _, _, image_names in os.walk(label_folder):
          print(label_folder)
          relative_image_names = []
          for image in image_names:
            relative_image_names.append( dirpath + "/"+dataset_fldr+'/image/'+ image)
          images_train.extend(relative_image_names)
          #print(relative_image_names)
    return images_train

image_path = create_img_path(dataset_path_train)
traindata_ = pd.DataFrame.from_dict({'image_path': image_path})




image_path = create_img_path(dataset_path_test)
testdata_ = pd.DataFrame.from_dict({'image_path': image_path})


# In[4]:



def generate_examples(example):
        path = example['image_path']
        image = Image.open(path)
        w,h = image.size
        filename = path.split('/')[-1]
        filename = filename.split('.')[0]
        datdir=path.split('/')[-3]
        #print(datdir)
        f = open(dirpath + '/' + datdir + '/changed/'+ filename + '.json')
        data = json.load(f)
        words = []
        boxes = []
        neighbors = []
        labels = []
        #print(data)
        for i in data:
          words.append(i['text'])
          unnormalized_box = i['box']
          normalized_box = normalize_box(i['box'],w,h)
          #normalized_box = unnormalized_box
          # if(i['text'] not in words_dictionary):
          #   global id
          #   words_dictionary[str(i['text'])] = id
          #   box_dictionary[id] = normalized_box
          #   id+=1
          boxes.append(normalized_box)
          neighbors.append(i['neighbors'])
          labels.append(label2idx[i['label']])
        # normalize the bounding boxes
        # add as extra columns 
        assert len(words) == len(boxes)
        example['words'] = words
        example['bbox'] = boxes
        example['neighbors'] = neighbors
        example['labels'] = labels
        return example

traindata = Dataset.from_pandas(traindata_)
train = traindata.map(generate_examples)
testdata = Dataset.from_pandas(testdata_)
test = testdata.map(generate_examples)


# In[5]:










{'menu.price': 0, 'menu.itemsubtotal': 1, 'menu.nm': 2, 'sub_total.service_price': 3, 'total.emoneyprice': 4, 'sub_total.etc': 5, 'total.changeprice': 6, 'total.total_etc': 7, 'menu.discountprice': 8, 'O': 9, 'sub_total.discount_price': 10, 'total.menutype_cnt': 11, 'total.creditcardprice': 12, 'menu.sub_nm': 13, 'menu.sub_cnt': 14, 'menu.num': 15, 'total.cashprice': 16, 'total.total_price': 17, 'total.menuqty_cnt': 18, 'sub_total.subtotal_price': 19, 'menu.cnt': 20, 'menu.unitprice': 21, 'sub_total.tax_price': 22, 'menu.sub_price': 23}
/home/pushpa/cord_original/train/image
/home/pushpa/cord_original/test/image


  0%|          | 0/800 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ex/s]

In [11]:
processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")



# we need to define custom features
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'labels': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),     
   
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'neighbours': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None)
    
})


# In[10]:


def preprocess_data(examples):
  #print(examples['image_path'])
  path = examples['image_path']
  images = Image.open(path).convert("RGB")
  w,h = images.size
  images = images.resize((224, 224))
  #images = np.asarray(images)  
  #images = images[:, :, ::-1] # flip color channels from RGB to BGR
  #images = images.transpose(2, 0, 1)
  words = examples['words']
  #print(examples['bbox'])
  bbox = [(normalize_box(x,w,h)) for x in examples['bbox'] ]
  #bbox = examples['bbox']
  #normalized_box = normalize_box(i['box'],w,h)
  word_labels = examples['labels']
  
  encoded_inputs = processor(images, words, boxes=bbox, word_labels=word_labels,
                             padding="max_length", truncation=True)
  
   

  #print( len(encoded_inputs['image']))
  del examples['words']
  del examples['image_path']
  #print(len(encoded_inputs['input_ids']))
   # get required padding length
  pad_len = 512 - len(examples['neighbors'])
  token_neighbors = [[[0]*6]*12]*pad_len
  #print(token_neighbors)
  examples['neighbors'] = examples['neighbors'] + token_neighbors
  
  dict1 = {'neighbors':list(examples['neighbors'])}
  #dict1 = {'neighbors':examples['neighbors']}
  encoded_inputs.update(dict1)
  encoded_inputs["image"] = np.array(encoded_inputs["image"])
  



  
  
  return encoded_inputs


# In[11]:



train_dataset = []
for example in train:
    #print(example)
    processed_example = preprocess_data(example)
    example.update(processed_example)
    train_dataset.append(example)



test_dataset = []
for example in test:
    #print(example)
    processed_example = preprocess_data(example)
    example.update(processed_example)
    test_dataset.append(example)

#encoding['neighbors'][0][0]


# In[ ]:


In [12]:
image_path

['/home/pushpa/cord_original/test/image/receipt_00099.png',
 '/home/pushpa/cord_original/test/image/receipt_00067.png',
 '/home/pushpa/cord_original/test/image/receipt_00088.png',
 '/home/pushpa/cord_original/test/image/receipt_00016.png',
 '/home/pushpa/cord_original/test/image/receipt_00019.png',
 '/home/pushpa/cord_original/test/image/receipt_00091.png',
 '/home/pushpa/cord_original/test/image/receipt_00017.png',
 '/home/pushpa/cord_original/test/image/receipt_00015.png',
 '/home/pushpa/cord_original/test/image/receipt_00014.png',
 '/home/pushpa/cord_original/test/image/receipt_00047.png',
 '/home/pushpa/cord_original/test/image/receipt_00095.png',
 '/home/pushpa/cord_original/test/image/receipt_00077.png',
 '/home/pushpa/cord_original/test/image/receipt_00039.png',
 '/home/pushpa/cord_original/test/image/receipt_00073.png',
 '/home/pushpa/cord_original/test/image/receipt_00096.png',
 '/home/pushpa/cord_original/test/image/receipt_00032.png',
 '/home/pushpa/cord_original/test/image/

In [ ]:
encoding = train_dataset[0]
print(processor.tokenizer.decode(encoding['neighbors'][0][0]))


# In[12]:


def collate_fn(batch):
   elem = batch[0]
   #print(type(elem))
   bbox = [item['bbox'] for item in batch]  # just form a list of tensor
   neighbours = [item['neighbors'] for item in batch]
   labels = [item['labels'] for item in batch]
   input_ids = [item['input_ids'] for item in batch]
   token_type_ids = [item['token_type_ids'] for item in batch]
   attention_mask = [item['attention_mask'] for item in batch]
   image =  [item['image'] for item in batch][0]
   elem= {'bbox' : torch.tensor(bbox), 'neighbors':torch.tensor(neighbours),'labels':torch.Tensor(labels),'input_ids':torch.Tensor(input_ids),'token_type_ids':torch.Tensor(token_type_ids),'attention_mask':torch.Tensor(attention_mask),'image':torch.Tensor(image)}
   #return dict(bbox = bbox, neighbors=neighbors,labels=labels,input_ids=input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask,image=image)
   #transposed = zip(*batch)
   #return [default_collate(samples) for samples in transposed]
   return elem



train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0,  collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=1,shuffle=True, num_workers=0,  collate_fn=collate_fn)






In [5]:
device = torch.device('cpu')

tokenizer = LayoutLMv2Tokenizer.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")
class LayoutLMv2ForTokenClassification(nn.Module):
    def __init__(self): 
        super().__init__()
        self.start_time = time.time()
        # LayoutLM base model + token classifier
        self.num_labels = len(label2idx)
        self.layoutlm = LayoutLMv2Model.from_pretrained("microsoft/layoutlmv2-base-uncased", num_labels=self.num_labels)
        
        self.mlp = nn.Linear(13*768,12*768)
        self.sigmoid = nn.Sigmoid()
        self.attn = Attention(768)

        self.dropout = nn.Dropout(self.layoutlm.config.hidden_dropout_prob)
        self.classifier = nn.Linear(2*768, self.num_labels)

    def forward(self,image,input_ids,bbox,attention_mask,token_type_ids,neighbors,inputs_embeds=None,position_ids=None,head_mask=None,
        labels=None,output_attentions=None,output_hidden_states=None,return_dict=None,):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the token classification loss. Indices should be in ``[0, ..., config.num_labels -
            1]``.

        """
        return_dict = return_dict if return_dict is not None else self.layoutlm.config.use_return_dict
        # print('layoutlm Start',time.time()-self.start_time)
        # first, forward pass on LayoutLM
        #tokenizer = LayoutLMv2Tokenizer.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")
        outputs = self.layoutlm(
            image=image,
            input_ids=input_ids,
            bbox=bbox,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds = inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        # print('layoutlm End',time.time()-self.start_time)
        # print(self.layoutlm.config.hidden_size)
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]
        sequence_output = outputs[0][:, :seq_length]
        dictionary = {}
        batchsize = sequence_output.shape[0]
        final_neighbors = []
        # print('Making Neighbours',time.time()-self.start_time)
        for batch in range(batchsize):
          for i in range(512):
            dictionary[int(input_ids[batch][i])] = sequence_output[batch][i]
          l = []
          for i in range(512):
            # for n in neighbors[batch]:
            temp = []
            temp = torch.Tensor(temp).cuda()
            for j in range(12):
              word_tokens = neighbors[batch][i][j]
              embed = torch.zeros(768).cuda()
              for w in word_tokens:
                w = w.item()
                if(w in dictionary):
                  # print(w)
                  embed = torch.add(embed,dictionary[w])
                else:
                  embed = torch.add(embed,torch.zeros(768).cuda())
              temp = torch.cat((temp,embed))
              
              # print('temp',temp.shape)
              # temp = torch.stack(temp)
            temp = torch.reshape(temp,(12,768))
            l.append(temp)
          l = torch.stack(l)
          final_neighbors.append(l)
        final_neighbors = torch.stack(final_neighbors)
        # print('final_neighbors',final_neighbors.shape)
        neighbors = final_neighbors.to(device)
        # neighbors = final_neighbors
        # print('Neighbours Made',time.time()-self.start_time)
        # print(neighbors.shape)
        # print('Gating and Attention Start',time.time()-self.start_time)
        final_output = []
        batchsize = sequence_output.shape[0]
        for i in range(batchsize): 
          temp = []
          for row,n in zip(sequence_output[i],neighbors[i]):
            # print(row.shape,n.shape)
            rij = []
            rij = torch.Tensor(rij)
            rij = rij.to(device)
            for z in range(12):
              # rij = torch.cat((row,n[z]),0)
              rij = torch.cat((rij,torch.cat((row,n[z]),0)),0)
            # print(rij.shape)
            wr = self.mlp(rij)
            # print(wr.shape)
            g = self.sigmoid(wr)
            # print(g.shape)
            g = torch.reshape(g,(12,768))
            c_dash = torch.mul(g,n)
            # print(c_dash.shape)
            c_dash = torch.reshape(c_dash,(12,768))
            c_dash = c_dash.unsqueeze(0)
            row = row.unsqueeze(0)
            row = row.unsqueeze(0)
            # print(row.shape)
            # print(c_dash.shape)
            output,weights = self.attn(row,c_dash)
            temp.append(output[0][0])
          temp = torch.stack(temp)
          # temp = .unsqueeze(0)
          # print(final_output.shape)
          final_output.append(temp)
        final_output = torch.stack(final_output)
        final_output = torch.cat((sequence_output,final_output),2)
        # print('Gating And Attention End',time.time()-self.start_time)
        # print(final_output.shape)
        # print('Classification Start',time.time()-self.start_time)
        final_output = self.dropout(final_output)
        logits = self.classifier(final_output)
        # attention_mask = torch.cat((attention_mask,attention_mask),1)
        # labels = torch.cat((labels,labels),1)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()

            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
        # print('Classification End',time.time()-self.start_time)
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        
        )


# In[13]:


from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)


In [6]:
from transformers import AdamW
import torch
from tqdm.notebook import tqdm

from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler,DataLoader,TensorDataset
import pprint
import numpy
from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)

# k_folds = 10
results = []
# kfold = KFold(n_splits=k_folds, shuffle=False)
num_train_epochs = 4
t_total = len(train_dataloader) * num_train_epochs # total number of training steps 
model = LayoutLMv2ForTokenClassification()
device = torch.device('cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=2.373260750776288e-05)
# wandb.init(name='First Run', 
#           project='FinalResultsWithFullDatasetwith45epochs',
#           notes='FinalResultsWithFullDataset', 
#           tags=['FinalResultsWithFullDataset'])
global_step = 0
# wandb.watch(model)
model.train() 
for epoch in range(num_train_epochs):  
  print("Epoch:", epoch)
  for batch in tqdm(train_dataloader):
        # input_ids = torch.Tensor(list(batch['input_ids'].values)).to(device)
        input_ids=batch['input_ids'].long().cpu()
        bbox=batch['bbox'].long().cpu()
        attention_mask=batch['attention_mask'].long().cpu()
        token_type_ids=batch['token_type_ids'].long().cpu()
        labels=batch['labels'].long().cpu()
        image = batch['image'].long().cpu()
        neighbors = batch['neighbors'].long().cpu()
        # print(batch)
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(labels=labels,
                      input_ids= input_ids,
                      bbox=bbox, 
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids,
                      image=image,
                      neighbors= neighbors)
        loss = outputs.loss
        # print loss every 100 steps
        if global_step % 100 == 0:
          print(f"Loss after {global_step} steps: {loss.item()}")
        loss.backward()
        optimizer.step()
        global_step += 1

save_path = f'/home/pushpa/cord.pth'
torch.save(model.state_dict(), save_path)

NameError: name 'train_dataloader' is not defined

In [ ]:
from datasets import load_metric
#torch.cuda.empty_cache()
model = LayoutLMv2ForTokenClassification()
#model = nn.DataParallel(model)
model.load_state_dict(torch.load(save_path), strict=False)
#device = torch.device('cuda:0')
#model = model.to(device)
metric = load_metric("seqeval")

# put model in evaluation mode
model.eval()
for batch in tqdm(test_dataloader, desc="Evaluating"):
    with torch.no_grad():
        input_ids=batch['input_ids'].long().cpu()
        bbox=batch['bbox'].long().cpu()
        attention_mask=batch['attention_mask'].long().cpu()
        token_type_ids=batch['token_type_ids'].long().cpu()
        labels=batch['labels'].long().cpu()
        image = batch['image'].long().cpu()
        neighbors = batch['neighbors'].long().cpu()

        # forward pass
        outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids, 
                        labels=labels,image=image,neighbors=neighbors)
        
        # predictions
        predictions = outputs.logits.argmax(dim=2)

        # Remove ignored index (special tokens)
        true_predictions = [
            [idx2label[p.item()] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [idx2label[l.item()] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

        metric.add_batch(predictions=true_predictions, references=true_labels)

final_score = metric.compute()
print(final_score)

In [ ]:
import numpy as np

preds_val = None
out_label_ids = None

# put model in evaluation mode
model.eval()
for batch in tqdm(test_dataloader, desc="Evaluating"):
    with torch.no_grad():
        input_ids=batch['input_ids'].long().cpu()
        bbox=batch['bbox'].long().cpu()
        attention_mask=batch['attention_mask'].long().cpu()
        token_type_ids=batch['token_type_ids'].long().cpu()
        labels=batch['labels'].long().cpu()
        image = batch['image'].long().cpu()
        neighbors = batch['neighbors'].long().cpu()

        # forward pass
        outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids, 
                        labels=labels,image=image,neighbors=neighbors)
        
        if preds_val is None:
          preds_val = outputs.logits.detach().cpu().numpy()
          out_label_ids = batch["labels"].detach().cpu().numpy()
        else:
          preds_val = np.append(preds_val, outputs.logits.detach().cpu().numpy(), axis=0)
          out_label_ids = np.append(
              out_label_ids, batch["labels"].detach().cpu().numpy(), axis=0
          )

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score)

def results_test(preds, out_label_ids, labels):
  preds = np.argmax(preds, axis=2)

  label_map = {i: label for i, label in enumerate(labels)}

  out_label_list = [[] for _ in range(out_label_ids.shape[0])]
  preds_list = [[] for _ in range(out_label_ids.shape[0])]

  for i in range(out_label_ids.shape[0]):
      for j in range(out_label_ids.shape[1]):
          if out_label_ids[i, j] != -100:
              out_label_list[i].append(label_map[out_label_ids[i][j]])
              preds_list[i].append(label_map[preds[i][j]])

  results = {
      "precision": precision_score(out_label_list, preds_list),
      "recall": recall_score(out_label_list, preds_list),
      "f1": f1_score(out_label_list, preds_list,average='weighted'),
  }
  return results, classification_report(out_label_list, preds_list)

In [ ]:
labels = ['O', 'S-ADDRESS', 'S-TOTAL', 'S-COMPANY', 'S-DATE']
val_result, class_report = results_test(preds_val, out_label_ids, labels)
print("Overall results:", val_result)
print(class_report)

In [ ]:
labels